In [1]:
import sys
sys.path.append('/home/dldx/UniRep/pipgcn')#sys.path.append('引用模块地址')
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from my_dataloader import GraphDataLoader, collate
from tqdm import tqdm
import random
import os
import time
import scipy.sparse as sp
import pickle
import dgl
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from dgl.nn.pytorch import GraphConv
from torch.utils.data import DataLoader

Using backend: pytorch


In [2]:
class Classifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes):
        super(Classifier, self).__init__()
        self.conv1 = GraphConv(in_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, hidden_dim)
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g,h):
        # Use node degree as the initial node feature. For undirected graphs, the in-degree
        # is the same as the out_degree.
        #h = g.in_degrees().view(-1, 1).float()
        # Perform graph convolution and activation function.
        h = F.relu(self.conv1(g, h))
        h = F.relu(self.conv2(g, h))
        g.ndata['h'] = h
        # Calculate graph representation by averaging all the node representations.
        hg = dgl.sum_nodes(g, 'h')
        return self.classify(hg)

In [3]:
# A4ZPX2
tied1_name_l = ['Q5G5V8','Q5G5V4','Q6EG58','A4ZPW4','P04578','Q4QXE8','A8CVW4','Q2MKA8','A1EAH8','A1EAI4','Q27Q69','Q70014','B2YFU9','B2YFQ5','O89960']
tied1_name_l.sort()
# A1EAI3
tied2_name_l = ['Q5G5U7','Q5G5U5','A7KVY7','Q5G5V1','B0FBI6','Q5G5V5','A1EAI2','A1EAI0','Q6TCV7','Q202K1','Q202K7','Q27Q74','A1EAI7','B2YFT0','Q8JDI3']
tied2_name_l.sort()
# A4ZPW9
tied3_name_l = ['A4ZPX1','A4ZPW7','A4ZPW8','Q5G5V2','Q5G5U6','A1EAG8','A1EAH7','A0MTL0','A1EAH6','A1EAH3','B2YFP6','B2YFP1','B2YFU4','B2YFV4','B2YFS0']
tied3_name_l.sort()
tied_name=tied1_name_l+tied2_name_l+tied3_name_l
with open("/home/dldx/UniRep/data/data45_true.p", 'rb') as f:                      
    dataset = pickle.load(f)

In [4]:
#trainset.num_classes
# set up seeds, args.seed supported
import numpy as np
seed=2021
torch.manual_seed(seed=seed)
np.random.seed(seed=seed)

#指定GPU
torch.cuda.set_device(1)
if torch.cuda.is_available():

    device = torch.device("cuda")
    torch.cuda.manual_seed_all(seed=seed)
else:
    device = torch.device("cpu")
print(device)
#device = torch.device("cpu")

cuda


In [5]:
# default collate function
def collate1(samples):
    # The input `samples` is a list of pairs (graph, label).
    graphs, labels,_ = map(list, zip(*samples))
    for g in graphs:
        # deal with node feats
        for key in g.node_attr_schemes().keys():
            g.ndata[key] = g.ndata[key].float()
        # no edge feats
    batched_graph = dgl.batch(graphs)
    labels = torch.tensor(labels)
    return batched_graph, labels

In [6]:
def train( net, trainloader, optimizer, criterion, epoch):
    net.train()
    running_loss = 0
    total_iters = len(trainloader)

    for graphs, labels in trainloader:
        # batch graphs will be shipped to device in forward part of model
        labels = labels.to(device)
        #feat = graphs.ndata['attr'].to(device)
        feat = graphs.ndata['feat'].float().to(device)
        output = net(graphs, feat)
#         outputs=output[0]
        loss = criterion(output, labels)
        running_loss += loss.item()

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # the final batch will be aligned
    running_loss = running_loss / total_iters

    return running_loss 

@torch.no_grad()
def eval_net( net, dataloader, criterion):
    net.eval()

    total = 0
    total_loss = 0
    total_correct = 0
    targets,outputs=[],[]
    for graphs, labels in dataloader:
        feat = graphs.ndata['feat'].float().to(device)
        labels = labels.to(device)
        total += len(labels)
        output = net(graphs, feat)
        _, predicted = torch.max(output.data, 1)

        total_correct += (predicted == labels.data).sum().item()
#         loss = criterion(output, labels)
#         total_loss += loss.item() * len(labels)
        targets.append(labels)
        outputs.append(output)
        acc = 1.0*total_correct / total
        
        
    return acc,targets,outputs

In [7]:
#模型参数
start = time.time()
datalist=[]
for fold_idx in range(10):
    model =Classifier(17, 64, 3).to(device)
    epochs=100
    criterion = nn.CrossEntropyLoss()  # defaul reduce is true
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)
    trainloader, validloader = GraphDataLoader(dataset, batch_size=45, device=device,
                                                collate_fn=collate1, seed=2021, shuffle=True,
                                                split_name='fold10', fold_idx=fold_idx).train_valid_loader()


#     valid_accs,targets,outputs=[],[],[]
    valid_acc_tmp=0
    for epoch in range(epochs):
        train( model, trainloader, optimizer, criterion, epoch)
        scheduler.step()

        
#---------------------------动态输出训练结果-----------------------------     
#         train_loss, train_acc = eval_net(model, trainloader, criterion)
#         train_losses.append(train_loss)
#         train_accs.append(train_acc)

       
        valid_acc,target,output = eval_net(model, validloader, criterion)
        targets=target
        outputs=output
        if valid_acc > valid_acc_tmp:
            valid_acc_tmp=valid_acc
        
    datalist.append([valid_acc_tmp,targets,outputs])
    print("第{}折完成,验证集准确率为{}".format(fold_idx,valid_acc_tmp)) 
    

loss_acc='gcn_acc_10fold_45.p'
with open(loss_acc, 'wb') as f:
    pickle.dump(datalist, f)
end = time.time()
print("运行时间:%.2f秒"%(end-start))      
print("work down!")

train_set : test_set = 40 : 5
第0折完成,验证集准确率为0.6
train_set : test_set = 40 : 5
第1折完成,验证集准确率为0.6
train_set : test_set = 40 : 5
第2折完成,验证集准确率为1.0
train_set : test_set = 40 : 5
第3折完成,验证集准确率为0.8
train_set : test_set = 40 : 5
第4折完成,验证集准确率为0.4
train_set : test_set = 41 : 4
第5折完成,验证集准确率为0.5
train_set : test_set = 41 : 4
第6折完成,验证集准确率为0.75
train_set : test_set = 41 : 4
第7折完成,验证集准确率为0.5
train_set : test_set = 41 : 4
第8折完成,验证集准确率为0.5
train_set : test_set = 41 : 4
第9折完成,验证集准确率为0.75
运行时间:89.11秒
work down!


In [25]:
#K folds result analyse

loss_acc='gcn_acc_10fold_45.p'
with open(loss_acc, 'rb') as f:
    all_info=pickle.load(f)

In [26]:
acc=[]
for i in range(10):
    acc.append(all_info[i][0])
print(np.mean(acc))

0.64


In [27]:
np.std(acc)

0.17291616465790582

In [20]:
acc

[0.6, 0.6, 1.0, 0.8, 0.4, 0.5, 0.75, 0.5, 0.5, 0.75]

In [24]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
all_rocauc=[]
AP=[]
for k in range(10):
    
    score=(all_info[k][2][0].cpu().numpy())
    label=(all_info[k][1][0].cpu().numpy())
    label=label_binarize(label, classes=[0, 1, 2])
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    y_test,y_score=label,score
    for i in range(3):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    print(roc_auc["micro"])
    all_rocauc.append(roc_auc["micro"])
    #=========================================
    Y_test,y_score=y_test,y_score
    n_classes=3
# For each class
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i],
                                                        y_score[:, i])
        average_precision[i] = average_precision_score(Y_test[:, i], y_score[:, i])

    # A "micro-average": quantifying score on all classes jointly
    precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
        y_score.ravel())
    average_precision["micro"] = average_precision_score(Y_test, y_score,
                                                     average="micro")
    AP.append(average_precision["micro"])
    print('Average precision score, micro-averaged over all classes: {0:0.2f}'
          .format(average_precision["micro"]))
print(np.mean(AP),np.std(AP))

0.8200000000000001
Average precision score, micro-averaged over all classes: 0.71
0.56
Average precision score, micro-averaged over all classes: 0.50
0.8800000000000001
Average precision score, micro-averaged over all classes: 0.80
0.68
Average precision score, micro-averaged over all classes: 0.58
0.48
Average precision score, micro-averaged over all classes: 0.41
0.625
Average precision score, micro-averaged over all classes: 0.50
0.5
Average precision score, micro-averaged over all classes: 0.40
0.28125
Average precision score, micro-averaged over all classes: 0.29
0.375
Average precision score, micro-averaged over all classes: 0.32
0.625
Average precision score, micro-averaged over all classes: 0.49
0.49922383172383167 0.15239209438945683


In [22]:
np.std(all_rocauc)

0.1759709226690592

In [23]:
np.mean(all_rocauc)

0.582625

In [ ]:
train_acc=[]
valid_acc=[]
for i in range(100):
    d=data[i*2+1].split()
    train_acc.append(float(d[1]))
    valid_acc.append(float(d[3]))  

In [ ]:
print(len(valid_acc))

In [ ]:
plt.plot(train_acc, color='cornflowerblue')
plt.plot(valid_acc, color='darkorange')
labels=['train_acc','valid_acc']
#plt.xlim([0, 50])
#plt.ylim([0, 1])
plt.xlabel('epoch')
plt.ylabel('acc')
plt.title('Train_acc vs Valid_acc')
plt.legend(labels,loc="lower right")
plt.savefig('T_V.png',format='png',dpi=300)